# R: Porównanie bibliotek
# Zbiór danych "cows"
---
## 1. Konstrukcja modeli i mierników parametrów
### 1.1 Biblioteki i edycja danych
#### > Biblioteki

In [1]:
library(dplyr)
library(tidyr)
library(lme4)
library(nlme)
library(mgcv)
library(profmem)

Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'tidyr' was built under R version 3.6.3"Warning message:
"package 'lme4' was built under R version 3.6.3"Loading required package: Matrix
Warning message:
"package 'Matrix' was built under R version 3.6.3"
Attaching package: 'Matrix'

The following objects are masked from 'package:tidyr':

    expand, pack, unpack

Warning message:
"package 'nlme' was built under R version 3.6.3"
Attaching package: 'nlme'

The following object is masked from 'package:lme4':

    lmList

The following object is masked from 'package:dplyr':

    collapse

Warning message:
"package 'mgcv' was built under R version 3.6.3"This is mgcv 1.8-31. For overview type 'help("mgcv-package")'.
Warning message:
"package 'profmem' was

#### > Wczytanie i edycja danych

In [2]:
data_set <- read.csv2("cows.csv")

cols <- c("cow.id", "btn3a1", "lactation")
data_set[cols] <- lapply(data_set[cols], factor)

head(data_set)

cow.id,btn3a1,lactation,milk,fat
1,1,1,7770,358
2,1,1,7341,376
3,1,1,6998,294
3,1,2,8564,331
3,1,3,8621,330
4,1,1,9536,365


### 1.2 Funkcje
#### > Funkcja tworząca modele

In [3]:
LMM <- function(model) {
  if(model == "lme4") {
    LMM_lme4 <- lmer(data=data_set, milk ~ btn3a1 + lactation + (1|cow.id))
    return(LMM_lme4)
  }
  else if(model == "nlme") {
    LMM_nlme <- lme(data = data_set, milk ~ btn3a1 + lactation, random = ~1|cow.id)
    return(LMM_nlme)
  }
  else if(model == "bam") {
    LMM_bam <- bam(data = data_set, milk ~ btn3a1 + lactation + s(cow.id, bs = "re"))
    return(LMM_bam)
  }
}

#### > Funkcja sprawdzająca czas

In [4]:
check_time <- function(model, n) {
  times <- c()

  for(i in 1:n) {
    start_time <- Sys.time()
    LMM(model)
    end_time <- Sys.time()
    times <- c(times, round(end_time - start_time, 4))
    }
    return(times)
  }

#### > Funkcja mierząca ilość zużytego RAMu

In [5]:
check_RAM <- function(model) {
  prof_mem <- profmem({
    LMM(model)
      })
  sum_in_MB <- round(sum(prof_mem$bytes[!is.na(prof_mem$bytes)]) * 1e-6, 4)
  return(sum_in_MB)
}

## 2. Symulacje
### 2.1 Podsumowanie modeli

#### > lme4

In [7]:
summary(LMM("lme4"))

Linear mixed model fit by REML ['lmerMod']
Formula: milk ~ btn3a1 + lactation + (1 | cow.id)
   Data: data_set

REML criterion at convergence: 17303.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.6325 -0.5378  0.0279  0.5505  3.2110 

Random effects:
 Groups   Name        Variance Std.Dev.
 cow.id   (Intercept) 1240403  1114    
 Residual             1252911  1119    
Number of obs: 1000, groups:  cow.id, 409

Fixed effects:
            Estimate Std. Error t value
(Intercept)  6699.45      81.08  82.632
btn3a12      -244.08     235.12  -1.038
lactation2   1307.04      84.63  15.443
lactation3   1800.54     102.10  17.635
lactation4   1669.27     176.45   9.460

Correlation of Fixed Effects:
           (Intr) btn312 lcttn2 lcttn3
btn3a12    -0.269                     
lactation2 -0.454  0.029              
lactation3 -0.374  0.015  0.395       
lactation4 -0.214  0.000  0.228  0.234

#### > nlme

In [8]:
summary(LMM("nlme"))

Linear mixed-effects model fit by REML
 Data: data_set 
       AIC      BIC    logLik
  17317.63 17351.95 -8651.814

Random effects:
 Formula: ~1 | cow.id
        (Intercept) Residual
StdDev:    1113.734 1119.335

Fixed effects: milk ~ btn3a1 + lactation 
               Value Std.Error  DF  t-value p-value
(Intercept) 6699.448  81.07595 588 82.63175  0.0000
btn3a12     -244.084 235.11696 407 -1.03814  0.2998
lactation2  1307.037  84.63442 588 15.44332  0.0000
lactation3  1800.538 102.10172 588 17.63475  0.0000
lactation4  1669.272 176.44866 588  9.46038  0.0000
 Correlation: 
           (Intr) btn312 lcttn2 lcttn3
btn3a12    -0.269                     
lactation2 -0.454  0.029              
lactation3 -0.374  0.015  0.395       
lactation4 -0.214  0.000  0.228  0.234

Standardized Within-Group Residuals:
        Min          Q1         Med          Q3         Max 
-2.63253348 -0.53779599  0.02789544  0.55054475  3.21103225 

Number of Observations: 1000
Number of Groups: 409 

#### > bam

In [9]:
summary(LMM("bam"))


Family: gaussian 
Link function: identity 

Formula:
milk ~ btn3a1 + lactation + s(cow.id, bs = "re")

Parametric coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  6699.45      81.08  82.632   <2e-16 ***
btn3a12      -244.08     235.12  -1.038      0.3    
lactation2   1307.04      84.63  15.443   <2e-16 ***
lactation3   1800.54     102.10  17.635   <2e-16 ***
lactation4   1669.27     176.45   9.460   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Approximate significance of smooth terms:
            edf Ref.df     F p-value    
s(cow.id) 277.4    407 2.351  <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

R-sq.(adj) =  0.577   Deviance explained = 69.6%
fREML = 8651.8  Scale est. = 1.2529e+06  n = 1000

#### > Z REML

In [7]:
model_lme4 <- fixef(LMM("lme4"))
model_nlme <- fixef(LMM("nlme"))
model_bam <- summary(LMM("bam"))$p.coeff

cbind(model_lme4, model_nlme, model_bam)

,model_lme4,model_nlme,model_bam
(Intercept),6699.4479,6699.4479,6699.4479
btn3a12,-244.0844,-244.0844,-244.0844
lactation2,1307.0367,1307.0367,1307.0367
lactation3,1800.5379,1800.5379,1800.5379
lactation4,1669.2723,1669.2723,1669.2723


### 2.2 Sprawdzenie czasu konstrukcji modeli
#### > 1000 razy z użyciem funkcji check_time(model, n)

In [11]:
LMM_lme4_times <- check_time("lme4", 100)
LMM_nlme_times <- check_time("nlme", 100)
LMM_bam_times <- check_time("bam", 100)

In [12]:
comb_DF <- data.frame(cbind(LMM_lme4_times, LMM_nlme_times, LMM_bam_times)) %>%
    gather(comb_DF, factor_key = TRUE) %>%
    group_by(comb_DF) %>%
    summarise(mean = mean(value), sd = sd(value), min = min(value), max = max(value))

comb_DF

`summarise()` ungrouping output (override with `.groups` argument)


comb_DF,mean,sd,min,max
LMM_lme4_times,0.028220,0.002500828,0.0250,0.0350
LMM_nlme_times,0.055059,0.004148825,0.0510,0.0729
LMM_bam_times,2.029584,0.219230436,1.8425,3.0582


### 2.3 Sprawdzenie zużytej pamięci RAM
#### > Z użyciem funkcji check_RAM(model)

In [13]:
lme4_RAM <- check_RAM("lme4")
nlme_RAM <- check_RAM("nlme")
bam_RAM <- check_RAM("bam")

cbind(lme4_RAM, nlme_RAM, bam_RAM)

lme4_RAM,nlme_RAM,bam_RAM
2.4157,3.663,326.9825


#### > Z użyciem wbudowanego w R narzędzia profilowania
https://rpubs.com/kamilpytlak/LMM_cows